In [ ]:
company = input(print("Enter Part Manufacturing company name to send mail:"))

Enter Part Manufacturing company name to send mail:
ADRICK MARINE CORP


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import smtplib
import ssl
import re  
import smtplib 
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import pandas as pd

xls = pd.ExcelFile('/content/drive/MyDrive/data.xlsx')
df = pd.read_excel(xls, 'Sheet1')
df_new = [pd.DataFrame(v) for k, v in df.groupby('Part Mfg')]

def find_email(data):
    regex = r"\"?([-a-zA-Z0-9.`?{}]+@\w+\.\w+)\"?"   
    l2=[]
    for k in data:
        for j in k:
            if(re.search(regex,str(j))):
                res = bool(re.search(r"\s", str(j)))
                if res == False:
                    l2.append(j)
    return l2

def send_mail(l2,data,df1):
    for x in l2: 
        try:
            msg = MIMEMultipart()
            msg['Subject'] = "Request for Quote - Reference " + str(data[0][1]) + "(OEM: " + str(company) +", CAGE: " + str(data[0][7]) + ")"
            msg['From'] = 'navneeths31@gmail.com'
            msg['To'] = str(x)
            html = """\
            <html>
              <head></head>
              <body>
                {0}
              </body>
            </html>
            """.format(df1.to_html())

            part1 = MIMEText(html, 'html')
            msg1 = "Hello,\n\nPlease provide pricing and availbility on the following:\n\n"
            msg2 = "\nSincere Thanks,\n\nPraveen Ramaswamy\n\nMEM Corporation\nUnits 114-115,4470 Regency Place\nWhite Plains, Maryland 20695\nTel: 1 301 861 3300\nFax: 1 301 363 1159"
            msg1 = MIMEText(msg1, 'plain')
            msg2 = MIMEText(msg2, 'plain')
            msg.attach(msg1)
            msg.attach(part1)
            msg.attach(msg2)
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login("navneeths31@gmail.com","knwz dwrs teju eekw")
            server.sendmail(msg['From'], msg['To'], msg.as_string())
            server.close()
            print ("Email sent successfully to", str(x)) 
        except Exception as ex: 
            print("Something went wrong....",ex)
            
for i in range(0,len(df_new)):
    l2= []
    data = df_new[i]
    pm = data['Part Mfg']
    data = data.values.tolist()
    pm = pm.to_string()
    if company in pm:
        l2 = find_email(data)
    if l2:
        df1 = pd.DataFrame(df_new[i])
        df1 = df1[['Part #','Part Desc.','Qty.','UI']].copy()
        df1 = df1.rename(columns = {'Part #':'PART #','Part Desc.':'DESCRIPTION','Qty.':'QTY','UI':'UOM'})
        df1 = df1.reset_index(drop=True)
        df1 = df1.rename(index = lambda x: x + 1)
        send_mail(l2,data,df1)
        break

Email sent successfully to navneeths31@gmail.com
Email sent successfully to nsreeni1@uncc.edu
